# GPU-Jupyter

This Jupyterlab Instance is connected to the GPU via CUDA drivers. In this notebook, we test the installation and perform some basic operations on the GPU.

## Test GPU connection

#### Using the following command, your GPU type and its NVIDIA-SMI driver version should be listed:

In [1]:
!nvidia-smi

Fri Dec 20 09:42:29 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.26       Driver Version: 440.26       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 207...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   54C    P0    38W / 215W |    204MiB /  7974MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

#### Now, test if PyTorch can access the GPU via CUDA:

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 891330946073693377, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9415777875944419380
 physical_device_desc: "device: XLA_CPU device"]

In [4]:
from __future__ import print_function
import numpy as np
import torch
a = torch.rand(5, 3)
a

tensor([[0.8722, 0.5115, 0.9504],
        [0.7723, 0.2860, 0.5793],
        [0.5388, 0.5681, 0.4295],
        [0.5269, 0.5165, 0.7475],
        [0.4882, 0.8255, 0.6498]])

## Performance test

#### Now we want to know how much faster a typical operation is using GPU. Therefore we do the same operation in numpy, PyTorch and PyTorch with CUDA. The test operation is the calculation of the prediction matrix that is done in a linear regression.

### 1) Numpy

In [5]:
x = np.random.rand(10000, 256)

In [6]:
%%timeit
H = x.dot(np.linalg.inv(x.transpose().dot(x))).dot(x.transpose())

590 ms ± 41.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2) PyTorch

In [7]:
x = torch.rand(10000, 256)

In [8]:
%%timeit
# Calculate the projection matrix of x
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

853 ms ± 16.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 3) PyTorch on GPU via CUDA

In [9]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 256, device=device) # directly create a tensor on GPU
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])
    print(y.to("cpu", torch.double)[0:5, 0:5])

tensor([[0.1054, 0.3291, 0.7729, 0.6005, 0.2372],
        [0.1022, 0.4534, 0.3964, 0.9174, 0.2610],
        [0.3969, 0.5472, 0.3876, 0.1979, 0.4063],
        [0.3630, 0.6374, 0.4176, 0.4804, 0.0396],
        [0.8256, 0.2289, 0.2265, 0.4388, 0.6070]], device='cuda:0')
tensor([[0.1054, 0.3291, 0.7729, 0.6005, 0.2372],
        [0.1022, 0.4534, 0.3964, 0.9174, 0.2610],
        [0.3969, 0.5472, 0.3876, 0.1979, 0.4063],
        [0.3630, 0.6374, 0.4176, 0.4804, 0.0396],
        [0.8256, 0.2289, 0.2265, 0.4388, 0.6070]], dtype=torch.float64)


In [10]:
%%timeit
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

11.3 ms ± 60.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Exhaustive Testing on GPU

In [11]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 10, device=device) # directly create a tensor on GPU

In [12]:
if torch.cuda.is_available():
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])

tensor([[0.3112, 0.7480, 0.1882, 0.8453, 0.8198],
        [0.5953, 0.8401, 0.3126, 0.6025, 0.5252],
        [0.1902, 0.5610, 0.7968, 0.1463, 0.7154],
        [0.7979, 0.2161, 0.6176, 0.2951, 0.1980],
        [0.6451, 0.3837, 0.5305, 0.2740, 0.3330]], device='cuda:0')


In [13]:
if torch.cuda.is_available():
    # Here is the memory of the GPU a border. 
    # A matrix with 100000 lines requires 37 GB, but only 8 GB are available.
    H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

In [14]:
if torch.cuda.is_available():
    print(H[0:5, 0:5])

tensor([[ 1.2748e-03,  5.3656e-04,  1.7376e-04,  3.3888e-06, -1.7049e-04],
        [ 5.3656e-04,  6.3624e-04,  2.5957e-05,  3.3281e-04, -1.6239e-05],
        [ 1.7376e-04,  2.5957e-05,  7.6328e-04,  7.7603e-05,  1.8272e-04],
        [ 3.3888e-06,  3.3281e-04,  7.7603e-05,  9.6281e-04,  1.2375e-04],
        [-1.7049e-04, -1.6239e-05,  1.8272e-04,  1.2375e-04,  6.9231e-04]],
       device='cuda:0')


In [15]:
if torch.cuda.is_available():
    # This operation is difficult, as an symmetric matrix is transferred 
    # back to the CPU. Is possible up to 30000 rows.
    print(H.to("cpu", torch.double)[0:5, 0:5])

tensor([[ 1.2748e-03,  5.3656e-04,  1.7376e-04,  3.3888e-06, -1.7049e-04],
        [ 5.3656e-04,  6.3624e-04,  2.5957e-05,  3.3281e-04, -1.6239e-05],
        [ 1.7376e-04,  2.5957e-05,  7.6328e-04,  7.7603e-05,  1.8272e-04],
        [ 3.3888e-06,  3.3281e-04,  7.7603e-05,  9.6281e-04,  1.2375e-04],
        [-1.7049e-04, -1.6239e-05,  1.8272e-04,  1.2375e-04,  6.9231e-04]],
       dtype=torch.float64)
